#### Градиентный бустинг на базе CatBoost (используя встроенный текстовый векторайзер)

In [2]:
import numpy as np
import pandas as pd

In [7]:
from utils import *

In [3]:
from catboost import Pool
from catboost import CatBoostClassifier

In [4]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_feather('data/data.ftr')
X, y = df.drop(columns='topic'), df['topic']
params = {'stratify': y, 'test_size': 0.2, 'random_state': 42}
X_train, X_valid, y_train, y_valid = train_test_split(X, y, **params)

* *загружаем предобработанный датафрейм*
* *делим данные на 2е части: 80% в обучении, стратификация по целевой переменной*

In [ ]:
cat_params = {
    'iterations': 1000, 
    # 'task_type': 'GPU', 
    'learning_rate': 0.1, 
    'eval_metric': 'MultiClass', 
    'loss_function': 'MultiClass', 
    'early_stopping_rounds': 10, 
    'use_best_model': True, 
    'random_seed': 42, 
    'verbose': 50
}

train_pool = Pool(
    X_train, y_train, 
    feature_names=list(X_train), 
    text_features=['title','text']
)

valid_pool = Pool(
    X_valid, y_valid, 
    feature_names=list(X_valid), 
    text_features=['title','text']
)

cat = CatBoostClassifier(**cat_params)
cat.fit(train_pool, eval_set=valid_pool)
y_pred = cat.predict(X_valid)

* *готовим данные для catboost, указываем текстовые фичи (catboost имеет свой встроенный векторайзер)*
* обучение на 80% данных заняло ~ 2ч на CPU (на GPU не пробовал, но поддержка для данного типа лосса должна быть)

In [6]:
accuracy_score(y_valid, y_pred)

0.9350009255831173

In [7]:
f1_score(y_valid, y_pred, average='macro')

0.9289541501418461

In [8]:
print(confusion_matrix(y_valid, y_pred))

[[ 7717   277   391   281    64   203]
 [  207 10205   222    76    17    32]
 [  427   288 25655   421    93   440]
 [  310   128   436  9613    10   130]
 [   64    29    84    20 12660    26]
 [  280    71   395   153    43 14964]]


In [9]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86      8933
           1       0.93      0.95      0.94     10759
           2       0.94      0.94      0.94     27324
           3       0.91      0.90      0.91     10627
           4       0.98      0.98      0.98     12883
           5       0.95      0.94      0.94     15906

    accuracy                           0.94     86432
   macro avg       0.93      0.93      0.93     86432
weighted avg       0.94      0.94      0.94     86432



* *на данный момент эта модель является лучшей с точки зрения качества*
* *опять же, видно, что хуже всего модель справляется с классом 0 (Интернет и СМИ)*

In [8]:
list(load_json('data/labels.json').keys())

['Интернет и СМИ', 'Культура', 'Мир', 'Наука и техника', 'Спорт', 'Экономика']

In [10]:
cat.save_model('data/catboost.bin')

* *размер файла модели ~20Мб*
* *внешние эмбеддинги не нужны (встроенный векторайзер)*